<a href="https://colab.research.google.com/github/ycc-br/Repo-1/blob/main/Exp_Colab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="files/image.pfng",width=800, height=400>

# INTRODUÇÃO

## EXPORTAÇÃO

Exportação, basicamente, consiste no ato de vender para outro país, tanto mercadorias quanto serviços.<br>
Para o Brasil é bom, pois contribui para criação e manutenção de emprego e renda, aumento da competitividade.<br>
Já para a empresa exportadora também é positivo, pois diminui o risco do negócio devido a diversificação geográfica, incenvtivos fiscais, aumento do mercado potencial. 


## OBJETIVO

Conhecer as exportações ao responder as seguintes questões:

Exportações brasileiras | 2021

1. Quem são os maiores compradores do Brasil em 2021? E o que eles mais compraram?
2. Quais mercadorias o Brasil mais vendeu ao exterior em 2021? (as commodities dominam mesmo?) E para quem o Brasil vendeu?

Exportações brasileiras | 2010 - 2021
1. Evolução das exportações ao longo do período (em valores transacionados)
2. Os maiores compradores do Brasil no período e o que eles compraram
3. Quais mercadorias o Brasil mais exportou no período e para quem vendeu?


In [ ]:
!pip install plotly==5.4.0

# IMPORTAÇÃO DAS LIBS 

In [ ]:
# Importando as libs necessárias
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive

pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# Estabelecendo a conexão com o Google Drive
drive.mount('/content/drive')

# OBTENÇÃO E TRATAMENTO DOS DADOS

# Nova seção

O dataset utilizado foi obtido do seguinte site: https://www.gov.br/produtividade-e-comercio-exterior/pt-br/assuntos/comercio-exterior/estatisticas/base-de-dados-bruta

In [ ]:
# Lendo o dataset
raw_data = pd.read_csv('EXP_COMPLETA.csv',sep=';')

# Por default, as 5 primeiras linhas
raw_data.head()

In [ ]:
# Renomeando as colunas
raw_data.columns = ['ANO','MES','NCM','UNI_MED','PAIS','UF','MODAL','URF','UNI_STAT','KG_LIQ','VALOR_FOB']
raw_data

**Dicionário das variáveis**

**ANO:** Ano (1997 a 2021)

**MES:** Mês 

**NCM:** Nomenclatura Comum Mercosul - Utilizada para controle e identificação das mercadorias comercializadas no Brasil e nos outros países do Mercosul (cada NCM representa uma mercadoria diferente)

**UNID:** Código da Unidade de Medida Estatística que é uma unidade de medida padrão para cada NCM, podendo ter valores como quilograma, metro, litro, pares, tonelada e outros.

**PAIS:** Código do país com a qual foi realizada a operação (importação ou exportação)

**UF:** Unidade Federativa (estado) de origem (exportação) ou destino (importação).

**MODAL:** Código para identificação do meio de transporte utilizado (aéreo, marítimo, rodoviária, ferroviária e outros). Na *exportação*, é o método utilizado para o transporte de mercadorias entre o último local de embarque para o exterior. Na *importação*, configura-se através dos meios de acesso para os bens do primeiro ponto de entrada no território nacional.

**URF (Unidade da Receita Federal):** Código da agência responsável pela execução dos procedimentos necessários para o desembaraço aduaneiro da mercadoria importada/exportada

**UNI_STAT:** No detalhamento por NCM, cada produto tem sua unidade estatística. Grande parte dos produtos tem como unidade estatística o peso em quilogramas, mas existem outras: quilograma líquido, número (unidades), pares, dúzias, milheiro, tonelada. A tabela completa que relaciona cada NCM com sua unidade estatística pode ser encontrada em na tabela “NCM _ UNIDADE”. É importante ressaltar que não se deve somar quantidades estatísticas de NCMs que contenham unidades estatísticas diferentes. 

**KG_LIQ:** Medida que expressa o peso líquido da mercadoria. Mesmo produtos com quantidades estatísticas diferentes do quilograma também possuem disponível a medida em quilograma, referindo-se ao peso líquido da mercadoria, ou seja, mercadoria desconsiderando embalagens, caixas ou quaisquer outros adicionais de transporte. 

**VALOR_FOB:** O valor FOB indica o preço da mercadoria em dólares americanos (US$) sob o Incoterm FOB (Free on Board), modalidade na qual o vendedor é responsável por embarcar a mercadoria enquanto o comprador assume o pagamento do frete, seguros e demais custos pós embarque.

*Vale relembrar que as informações contidas no dataset relativo às operações de comércio exterior, é de livre preenchimento e de responsabilidade exclusiva dos operadores de comércio exterior.*


In [ ]:
# Obtendo os anos abrangidos pelo dataset
raw_data.ANO.unique()

**<font size="4"> Selecionar os dados de interesse </font>**

O dataset abrange o período de 1997 a 2021. O escopo da presente análise será de 2010 a 2021.

In [ ]:
# Criando um subset - de 2010 até 2021
exp = raw_data[raw_data['ANO']>=2010].copy()
exp.reset_index(inplace=True)
exp.drop(['index'], axis=1, inplace=True)
exp

In [ ]:
# Selecionando as colunas de interesse para a análise
columns = ['ANO', 'MES', 'NCM', 'PAIS', 'MODAL','VALOR_FOB']
exp = exp.copy()
exp = exp[columns]
exp

**<font size="4"> Substituição dos códigos das colunas *Modais* e *País* pela respectiva descrição -  </font>**

<font size="4"> Modais </font> 

In [ ]:
# Lendo o CSV contendo o código e a respectiva descrição
cod_modais = pd.read_csv('CO_VIA.csv',sep=';')

# Criando dicionário com os códigos e respectiva descrição
modal_dict = {cod_modais.iloc[i,0]=cod_modais.iloc[i,1] for i in range(17)}

# Substituindo o código pela respectiva descrição no dataframe
exp['MODAL'] = exp['MODAL'].map(modal_dict)

# Apresentando o dataframe
exp

<font size="4"> Países </font> 

In [ ]:
# Lendo o CSV contendo o código e a respectiva descrição
cod_pais = pd.read_csv('CO_PAIS.csv',sep=';')

# Criando dicionário com os códigos e respectiva descrição
pais_dict = {cod_pais.iloc[i,0]=cod_pais.iloc[i,3] for i in range(281)}
    
# Substituindo o código pela respectiva descrição no dataframe
exp['PAIS']=exp['PAIS'].map(pais_dict)

# Apresentando o dataframe
exp

**<font size="4"> Ajuste do tipo das variáveis para otimizar o uso de memória </font>**

In [ ]:
# Identificando o tipo de variável de cada coluna
exp.info()

In [ ]:
# Ajustando o tipo de variável
exp['ANO'] = exp['ANO'].astype('int16')
exp['MES']=exp['MES'].astype('int8')
exp['NCM']=exp['NCM'].astype('int32')
exp['VALOR_FOB']=exp['VALOR_FOB'].astype('uint32')

In [ ]:
# Redução de 273 MB, deixando o dataframe mais leve!
exp.info()

**<font size="4"> Valores ausentes e inconsistentes </font>**

In [ ]:
# Checando a existência de missing values
# A contagem de todas as colunas estão idênticas, indicando que não há valores faltantes
exp.describe(include='all')

In [ ]:
# Qtde e porcentagem de registros (linhas) a serem excluídas (mercadorias sem valor comercial)
print(f"Linhas a serem excluídas: {exp[exp['VALOR_FOB']==0].shape[0]}")
print(f"Equivalente a: {exp[exp['VALOR_FOB']==0].shape[0]/exp.shape[0]:.3f}%")

# Excluindo os registros de mercadorias sem valor comercial 
exp = exp.drop(exp[exp['VALOR_FOB']==0].index, axis=0)


In [ ]:
# Salvando o dataset limpo e tratado para quaisquer eventualidade
exp.to_csv('exp_tratado.csv', index=False)

<br>**<font size='5'> Análise Exploratória dos Dados (EDA)</font>**

Nesta etapa, iremos analisar o dataset para responder aos questionamentos sobre o panorama das exportações brasileiras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
exp = pd.read_csv('/content/drive/My Drive/exp_tratado.csv')
exp

In [ ]:
exp.info()

In [ ]:
# Iremos direcionar a atenção ao ano corrente (2021)
exp_2021 = exp[exp['ANO']==2021].copy()

In [ ]:
# O dataset possui registros atualizados até outubro de 2021
exp_2021['MES'].unique()

**<font size='4'> 1. Quais países são os maiores compradores? </font>** <font size="3"> (em termos de valor_fob) </font> 

In [ ]:
paises = pd.DataFrame(exp_2021.groupby(['PAIS'])[['VALOR_FOB']].sum())
paises

In [ ]:
# Agrupando os dados por país
paises = pd.DataFrame(exp_2021.groupby(['PAIS'])[['VALOR_FOB']].sum())

# Calculando a participação de cada país nas exportações brasileiras
paises['%']=paises['VALOR_FOB']/paises['VALOR_FOB'].sum()*100

# Ordenando os países em ordem decrescente
paises.sort_values(['%'],ascending=False,inplace=True)

# Calculando a participação acumulada para fins de análise
paises_acu = [32.83]
for i in range(239):
    paises_acu.append(paises_acu[i]+paises.iloc[i+1,1])
paises['% ACU']=paises_acu

# Formatando os valores de transação com separador de milhares
paises['VALOR_FOB']=paises['VALOR_FOB'].apply(lambda x: "{:_.2f}".format(x).replace('.',',').replace('_', '.'))

# Apresentando os 10 primeiros colocados
paises.head(10)

Em **2021**, houve concentração em 3 países **(China, Estados Unidos e Argentina)**, que responderam por quase **metade** das exportações.
<BR>Vale ressaltar que a **China, sozinha**, respondeu por **1/3** das exportações brasileiras. **3x** mais do que os **EUA**. 

In [ ]:
# Plotando os dados
# O tamanho dos quadros é proporcional ao valor transacionado do país em questão

paises['VALOR_FOB'] = paises['VALOR_FOB'].str.replace('.','').str.replace(',','').astype(float)
fig = px.treemap(paises, path=[px.Constant('Mundo'),paises.index], values=paises['VALOR_FOB'],color_discrete_sequence=px.colors.qualitative.Set3)
fig.update_layout(title={'text':'Participação dos países nas exportações brasileiras','y':0.95,'x':0.5})
fig.show()

<br>Agora vamos conhecer a lista de compras dos TOP 3 - China, EUA e Argentina

<font size='4'> 2. O que os TOP 3 mais compram do Brasil?</font>

<font size='5'>NCM</font>     significa Nomenclatura Comum do Mercosul. Basicamente, é um código de 8 dígitos que representa uma mercadoria. 
<br> E existe uma tabela que faz a correspodência de cada NCM com a respectiva mercadoria chamada *Tabela NCM*
<br>Por exemplo: 
<br> *0101.21.00* - Cavalos reprodutores de raça pura
<br> *0402.21.10* - Leite integral 
<br><br>Para conhecer em mais detalhes: https://www.fazcomex.com.br/blog/tabela-ncm/

In [ ]:
# Lendo CSV com a tabela NCM
ncm = pd.read_excel('/content/drive/My Drive/NCM completa x.xlsx')

# Criando dicionário com o conteúdo da tabela NCM
ncm_dict={ncm.iloc[i,0]:ncm.iloc[i,1] for i in range(ncm.shape[0])}
ncm_dict

<font size='4'> Rank 1 - China  </font>

In [ ]:
# Filtrando por país e agrupando por NCM
chn = exp_2021[exp_2021['PAIS'].isin(['China'])].groupby(['NCM'])[['VALOR_FOB']].sum()

# Calculando a participação de cada mercadoria
chn['%']=chn['VALOR_FOB']/chn['VALOR_FOB'].sum()*100

# Ordenando as mercadorias em ordem descrescente
chn = chn.sort_values(['VALOR_FOB'], ascending=False)

# Calculando a participação acumulada para fins de análise
acu = [chn.iloc[0,1]]
for i in range(chn.shape[0]-1):
    acu.append(acu[i]+chn.iloc[i+1,1])
chn['% Acu'] = acu

# Acrescentando coluna com a descrição da mercadoria para cada NCM
chn['NCM-Descrição']=chn.index.map(ncm_dict)

# Resetando o index
chn = chn.reset_index()

# Formatando os valores de transação com separador de milhares
chn['VALOR_FOB']=chn['VALOR_FOB'].apply(lambda x: "{:_.2f}".format(x).replace('.',',').replace('_', '.'))

# Apresentando os 10 primeiros colocados
chn.head(10)

As compras da China estão concentradas em: **Minério de ferro, Soja e Petróleo**.
<br>As 3 mercadorias, isoladas, correspondem por praticamente **80%** de tudo o que a China compra do Brasil

In [ ]:
# Plotando a lista de compras da China
# O tamanho dos quadros é proporcional ao valor transacionado da mercadoria em questão

chn['VALOR_FOB'] = chn['VALOR_FOB'].str.replace('.','').str.replace(',','').astype(float)
fig = px.treemap(chn, path=['NCM-Descrição'], values=chn['VALOR_FOB'],color_discrete_sequence=px.colors.qualitative.Set3)
fig.update_layout(title={'text':'LISTA DE COMPRAS - CHINA','y':0.925,'x':0.5})
fig.show()

Como já temos um exemplo do algoritmo para **gerar a lista de compra**, vamos criar uma **classe** para **automatizar** o trabalho.

In [ ]:
# Definindo uma classe - lista de compras de cada país

class pauta_pais:
    
    def __init__(self,df,ncm_dict,pais,linhas):
        self.df = df
        self.ncm_dict = ncm_dict
        self.pais = pais
        self.linhas = linhas

    def lista (self): # Este método produz a lista de compras com os valores transacionados
        
        # Filtrando por país e agrupando por NCM
        pauta = self.df[self.df['PAIS'].isin([self.pais])].groupby(['NCM'])[['VALOR_FOB']].sum()

        # Calculando a participação de cada mercadoria
        pauta['%']=pauta['VALOR_FOB']/pauta['VALOR_FOB'].sum()*100

        # Ordenando as mercadorias em ordem descrescente
        pauta = pauta.sort_values(['VALOR_FOB'], ascending=False)
        
        # Calculando a participação acumulada para fins de análise
        acu = [pauta.iloc[0,1]]
        for i in range(pauta.shape[0]-1):
            acu.append(acu[i]+pauta.iloc[i+1,1])
        pauta['% ACU'] = acu

        # Acrescentando coluna com a descrição da mercadoria para cada NCM
        pauta['DESCRIÇÃO']=pauta.index.map(self.ncm_dict)

        # Resetando o index
        pauta = pauta.reset_index()

        # Alterando sequência das colunas
        pauta = pauta[['NCM', 'DESCRIÇÃO','VALOR_FOB','%','% ACU']]
        
        # Formatando os valores de transação com separador de milhares
        pauta['VALOR_FOB']=pauta['VALOR_FOB'].apply(lambda x: "{:_.2f}".format(x).replace('.',',').replace('_', '.'))
        
        # Apresentando os primeiros colocados
        return pauta.head(self.linhas)
    
    
    def tree(self): # Este método plota a lista de compras do país-parâmetro
        
        # Filtrando por país e agrupando por NCM
        pauta = self.df[self.df['PAIS'].isin([self.pais])].groupby(['NCM'])[['VALOR_FOB']].sum()

        # Calculando a participação de cada mercadoria
        pauta['%']=pauta['VALOR_FOB']/pauta['VALOR_FOB'].sum()*100

        # Ordenando as mercadorias em ordem descrescente
        pauta = pauta.sort_values(['VALOR_FOB'], ascending=False)
        
        # Calculando a participação acumulada para fins de análise
        acu = [pauta.iloc[0,1]]
        for i in range(pauta.shape[0]-1):
            acu.append(acu[i]+pauta.iloc[i+1,1])
        pauta['% ACU'] = acu

        # Acrescentando coluna com a descrição da mercadoria para cada NCM
        pauta['DESCRIÇÃO']=pauta.index.map(self.ncm_dict)

        # Resetando o index
        pauta = pauta.reset_index()

        # Alterando sequência das colunas
        pauta = pauta[['NCM', 'DESCRIÇÃO','VALOR_FOB','%','% ACU']]
        
        # Plotando a lista de compras do país em questão
        fig = px.treemap(pauta, path=['NCM'], values=pauta['%'],color_discrete_sequence=px.colors.qualitative.Set3)
        fig.update_layout(title={'text':f'LISTA DE COMPRAS - {self.pais.upper()}','y':0.925,'x':0.5})
        fig.show()

<font size='4'> Rank 2 - EUA </font>

In [ ]:
# Criando uma nova instãncia da classe e informando os parâmetros necessários
eua = pauta_pais(exp_2021, ncm_dict,'Estados Unidos',10)

# Listando as compras dos EUA
eua.lista()

Contrastando com a China, a pauta norte-americana é mais distribuída. O TOP 10 não alcança metade das compras.
<br> Porém, o TOP 2 (**Semimanufaturados de ferro e Petróleo**) representa praticamente **1/4** das compras norte-americanas

O gráfico também evidencia uma melhor distribuição das mercadorias

In [ ]:
# Plotando a lista de compras dos EUA
# O tamanho dos quadros é proporcional ao valor transacionado

eua.tree()

<font size='4'> Rank 3 - Argentina (los hermanos :p) </font>

In [ ]:
# Atribuindo Argentina como uma nova instãncia e informando os parâmetros necessários
arg = pauta_pais(exp_2021, ncm_dict,'Argentina',10)

# Listando as compras da Argentina
arg.lista()

In [ ]:
# Plotando a lista de compras da Argentina
# O tamanho dos quadros é proporcional ao valor transacionado

arg.tree()

A lista de compras da Argentina, em relação a China e EUA, não está concentrada em poucas mercadorias, o que é evidenciado no gráfico.

<font size='4'> Rank 4 - Países Baixos (Holanda) </font>

In [ ]:
# Atribuindo Argentina como uma nova instãncia e informando os parâmetros necessários
nhl = pauta_pais(exp_2021,ncm_dict, "Países Baixos (Holanda)", 10)

# Listando as compras dos Países Baixos
nhl.lista()

Na lista de compras dos Países Baixos, percebe-se uma concentração em 5 mercadorias, que dominam **50%** das compras holandesas

In [ ]:
# Plotando a lista de compras dos Países Baixos
# O tamanho dos quadros é proporcional ao valor transacionado
nhl.tree()

Este domínio também é evidenciado no gráfico. As **top 5** ocupam **metade** da área do gráfico. 

<font size='4'> Rank 5 - Chile </font>

In [ ]:
# Atribuindo Argentina como uma nova instãncia e informando os parâmetros necessários
chl = pauta_pais(exp_2021,ncm_dict,'Chile',10)

# Listando as compras do Chile
chl.lista()

Quase 30% do que o Chile compra do Brasil é petróleo! 

In [ ]:
# Plotando a lista de compras do Chile
# O tamanho dos quadros é proporcional ao valor transacionado

chl.tree()

Agora que já sabemos quem são os maiores compradores do Brasil e o que eles compram, vamos identificar quais são as mercadorias mais vendidas

**<font size='4'> 2. As mercadorias mais vendidas </font>** 

In [ ]:
# Agrupando os dados de 2021 por NCM e ordenando pelo valor transacionado
ncm_exp = exp_2021.groupby(['NCM'])[['VALOR_FOB']].sum().sort_values(['VALOR_FOB'], ascending=False)

# Calculando a participação de cada mercadoria na exportação brasileira
ncm_exp['%']=ncm_exp['VALOR_FOB']/ncm_exp['VALOR_FOB'].sum()*100

# Calculando a participação acumulada das mercadorias 
acu = [ncm_exp.iloc[0,1]]
for i in range(ncm_exp.shape[0]-1):
    acu.append(acu[i]+ncm_exp.iloc[i+1,1])
ncm_exp['% ACU'] = acu

# Incluindo a descrição de cada mercdoria
ncm_exp['DESCRIÇÃO'] = ncm_exp.index.map(ncm_dict)

# Resetando o index
ncm_exp.reset_index(inplace=True)

# Alterando sequência das colunas
ncm_exp = ncm_exp[['NCM','DESCRIÇÃO','VALOR_FOB','%','% ACU']]

# Formatando os valores de transação com separador de milhares
ncm_exp['VALOR_FOB']=ncm_exp['VALOR_FOB'].apply(lambda x: "{:_.2f}".format(x).replace('.',',').replace('_', '.'))

# Apresentando as 10 mercadorias mais vendidas
ncm_exp.head(10)

Podemos inferir que há um predomínio de commodities, com baixo valor agregado na pauta de exportação de 2021

In [ ]:
# Plotando a lista de compras do Chile
# O tamanho dos quadros é proporcional ao valor transacionado

fig = px.treemap(ncm_exp, path=['NCM'], values=ncm_exp['%'],color_discrete_sequence=px.colors.qualitative.Set3)
fig.update_layout(title={'text':'Mercadorias mais vendidas pelo Brasil','y':0.925,'x':0.5})
fig.show()

<br>Agora vamos ver quem são os **maiores compradores** das principais commodities!

Para facilitar nossa análise, aproveitaremos o código da classe *pauta_pais* e criar uma nova classe *ncm_pais*.<br> Esta classe irá listar os países importadores da mercadoria sob análise

In [ ]:
# Criando a classe
class ncm_pais:
    
    def __init__(self,df,ncm,descricao,linhas):
        self.df = df
        self.ncm_dict = ncm_dict
        self.ncm = ncm
        self.linhas = linhas
        self.descricao = descricao

    def lista (self):
        
        # Filtrando por país e agrupando por NCM
        df = self.df[self.df['NCM'].isin([self.ncm])].groupby(['PAIS'])[['VALOR_FOB']].sum()

        # Calculando a participação de cada mercadoria
        df['%']=df['VALOR_FOB']/df['VALOR_FOB'].sum()*100

        # Ordenando as mercadorias em ordem descrescente
        df = df.sort_values(['VALOR_FOB'], ascending=False)
        
        # Calculando a participação acumulada para fins de análise
        acu = [df.iloc[0,1]]
        for i in range(df.shape[0]-1):
            acu.append(acu[i]+df.iloc[i+1,1])
        df['% ACU'] = acu

        # Resetando o index
        df = df.reset_index()
        
         # Alterando sequência das colunas
        df = df[['PAIS','VALOR_FOB','%','% ACU']]
        
        # Formatando os valores de transação com separador de milhares
        df['VALOR_FOB']=df['VALOR_FOB'].apply(lambda x: "{:_.2f}".format(x).replace('.',',').replace('_', '.'))

        # Apresentando os n primeiros colocados
        return df.head(self.linhas)
    
    
    def tree(self):
        
        # Filtrando por país e agrupando por NCM
        df = self.df[self.df['NCM'].isin([self.ncm])].groupby(['PAIS'])[['VALOR_FOB']].sum()

        # Calculando a participação de cada mercadoria
        df['%']=df['VALOR_FOB']/df['VALOR_FOB'].sum()*100

        # Ordenando as mercadorias em ordem descrescente
        df = df.sort_values(['VALOR_FOB'], ascending=False)
        
        # Calculando a participação acumulada para fins de análise
        acu = [df.iloc[0,1]]
        for i in range(df.shape[0]-1):
            acu.append(acu[i]+df.iloc[i+1,1])
        df['% ACU'] = acu

        # Resetando o index
        df = df.reset_index()

        # Alterando sequência das colunas
        df = df[['PAIS','VALOR_FOB','%','% ACU']]
        
        # Plotando a participação de cada país nas exportações da mercadoria em questão
        fig = px.treemap(df, path=['PAIS'], values=df['%'],color_discrete_sequence=px.colors.qualitative.Set3)
        fig.update_layout(title={'text':f'O QUANTO CADA PAÍS COMPROU DE {self.descricao.upper()}','y':0.925,'x':0.5})
        fig.show()


<font size='4'> Rank 1 - Minério de ferro </font>

In [ ]:
# Criando uma nova instância e informando os parâmetros necessários
ferro = ncm_pais(exp_2021,26011100,'Minério de ferro',10)

# Listando os maiores países compradores de minério de ferro
ferro.lista()

O maior comprador do minério de ferro brasileiro até outubro de 2021 é, sem grande surpresa, a **China**. Ela comprou **70%** de todo o minério de ferro exportado. 

In [ ]:
# Plotando a  participação de cada país na exportação de minério de ferro
# O tamanho dos quadros é proporcional ao valor transacionado
ferro.tree()

Visualmente, podemos ver o peso da China na exportação de minério de ferro. 

<font size='4'> Rank 2 - Soja </font>

In [ ]:
# Criando uma nova instância e informando os parâmetros necessários
soja = ncm_pais(exp_2021,12019000,'Soja',10)

# Listando os maiores países compradores de soja
soja.lista()

O mesmo acontece com a soja. A **China**, um único país dentre os 244 analisados, comprou praticamente **70%** da soja exportada pelo Brasil

In [ ]:
# Plotando a  participação de cada país na exportação de minério de ferro
# O tamanho dos quadros é proporcional ao valor transacionado
soja.tree()

<font size='4'> Rank 3 - Petróleo </font>

In [ ]:
# Criando uma nova instância e informando os parâmetros necessários
petroleo = ncm_pais(exp_2021,27090010,'Petróleo',10)

# Listando os maiores países compradores de petróleo
petroleo.lista()

Com o petróleo, a China ocupa novamente o topo. Mas *maneirou* no apetite. Importou *apenas* **quase a metade** do petróleo exportado pelo Brasil. <br> O segundo maior comprador, **EUA**, comprou menos de **1/4** do que a China comprou.

In [ ]:
# Plotando a  participação de cada país na exportação de minério de ferro
# O tamanho dos quadros é proporcional ao valor transacionado
petroleo.tree()

**<font size="4.5"> Evolução das exportações brasileiras | 2010 - 2021 </font>**

**<font size="4"> Maiores compradores do Brasil </font>**

In [ ]:
# Agrupando o dataset por ano e país 
exp_anual = exp.groupby(['ANO','PAIS'])[['VALOR_FOB']].sum().reset_index()

# Criando uma lista com os 5 maiores compradores de cada ano
lista_paises =[]
for i in exp_anual.ANO.unique():
     for j in range(5):
        lista_paises.append(exp_anual[exp_anual['ANO']==i].sort_values(['VALOR_FOB'], ascending=False).iloc[j,1])

# Criando um dataframe com os 5 maiores compradores de cada ano
rank_paises = pd.DataFrame()
count=0
for ano in exp_anual.ANO.unique():
    rank_paises[ano] = lista_paises[count:count+5]
    count+=5
    
# Apresentando o dataframe
rank_paises = rank_paises.T
rank_paises.columns = ['Rank 1','Rank 2','Rank 3','Rank 4','Rank 5'] 
rank_paises

Nos últimos 11 anos, **China, EUA, Argentina e Países Baixos** dominam os primeiros lugares. E no 5o lugar, há uma alternância entre Alemanha, Japão, Chile e Canadá. 
<br> E quanto os Top 5 transacionaram neste período?

In [ ]:
# Cruzando os dados das colunas PAIS e ANO
evol_paises = pd.crosstab(exp['PAIS'],exp['ANO'],values=exp['VALOR_FOB'],aggfunc=np.sum).T

# Selecionando os países em análise
evol_paises = evol_paises[['China','Estados Unidos','Argentina','Países Baixos (Holanda)','Alemanha','Japão','Chile','Canadá']]

# Formatando os valores transacionados 
for ano in evol_paises.columns:
    a[ano]=  a[ano].apply(lambda x: "{:_.2f}".format(x).replace('.',',').replace('_', '.'))

#  Apresentando os dados
evol_paises

O mais interessante é a **China**, além de permanecer em primeiro lugar, o volume de **compras** mais que **dobrou**. Enquanto que os demais países, as compras ou permaneceram estáveis ou houve um decréscimo. 

In [ ]:
# Criando lista com os países de interesse
paises = ['China','Estados Unidos','Argentina','Países Baixos (Holanda)','Alemanha','Japão','Chile','Canadá']

# Filtrando por países de interesse
line = exp[exp['PAIS'].isin(paises)]

# Agrupando o dataframe por ano e país
line = line.groupby(['ANO','PAIS'],as_index=False)[['VALOR_FOB']].sum()

# Plotando a evolução das compras dos países de interesse
fig = px.line(line, x="ANO", y="VALOR_FOB", color='PAIS',color_discrete_sequence=px.colors.qualitative.Set3)
fig.update_layout(title={'text':'EVOLUÇÃO - 2010 a 2021','y':0.925,'x':0.3})
fig.show()

O gráfico acima evidencia ainda mais o crescimento espantoso da China!

In [ ]:
fig = px.line(af.groupby(['ANO'])[['VALOR_FOB']].sum())
fig.show()

**<font size="4"> As mercadorias mais vendidas </font>**

In [ ]:
# Agrupando o dataset por ano e ncm
exp_anual = exp.groupby(['ANO','NCM'])[['VALOR_FOB']].sum().reset_index()

# Criando uma lista com as 5 mercadorias mais vendidas em cada ano
lista_ncm =[]
for i in exp_anual.ANO.unique():
     for j in range(5):
        lista_ncm.append(exp_anual[exp_anual['ANO']==i].sort_values(['VALOR_FOB'], ascending=False).iloc[j,1])

# Armazenando a lista em um dataframe
rank_ncm = pd.DataFrame()
count=0
for ano in exp_anual.ANO.unique():
    rank_ncm[ano] = lista_ncm[count:count+5]
    count+=5
    
# Apresentando o dataframe
rank_ncm = rank_ncm.T
rank_ncm.columns = ['Rank 1','Rank 2','Rank 3','Rank 4','Rank 5'] 
rank_ncm

Nada intituitivo a tabela acima certo? Vamos fazer a substituição do NCM pela respectiva mercadoria

In [ ]:
# Criando uma lista excluindo os NCMs repetidos
unique_ncm =[]
for ncm in lista_ncm:
    if ncm not in unique_ncm:
        unique_ncm.append(ncm)
        
# Colocando os NCM'S em um dataframe e incluindo a respectiva descrição
df_ncm = pd.DataFrame(lista_unica, columns=['NCM'])
df_ncm['DESCRIÇÃO'] = df_ncm.NCM.map(ncm_dict)
df_ncm

In [ ]:
# Incluir uma descrição simplificada das mercadorias
descr_simp = ['Minério ferro ñ aglomerado','Petróleo bruto','Outros grãos de soja','Açúcar de cana','Minério ferro aglomerado','Soja','Outros açúcares de cana','Bagaço da soja','Café em grão','Madeira (Pasta química)', 'Milho em grão','Carne bovina desossada']
df_ncm['DESCRIÇÃO SIMPLIFICADA']=descr_simp

# Criando dicionário relacionando os NCM's com a respectiva descrição simplificada
dict_ncm_simp = {df_ncm.iloc[i,0]:df_ncm.iloc[i,2] for i in range(12)}

# Substituindo os ncms pela respectiva descrição simplificada
for column in rank_ncm.columns:
    rank_ncm[column]=rank_ncm[column].map(dict_ncm_simp)
rank_ncm

Pronto! Agora ficou mais fácil de analisar. <br>
**Minério de ferro (ñ agloremado) e Soja** alternam o posto de **mercadoria mais vendida** pelo Brasil desde o ínicio do período analisado. Ou seja, **11 anos**. <br>
Em **2o**, o posto é compartilhado pelo **minério de ferro, soja e petróleo**. 
Vale ressaltar que o ranking acima evidencia novamente a **predominância de commodities** na pauta de exportação brasileira.


In [ ]:
# Criando um dataset apenas com as mercadorias mais exportadas no período
line = exp[exp['NCM'].isin(lista_unica)]

# Agrupando por ANO e NCM
line = line.groupby(['ANO','NCM'],as_index=False)[['VALOR_FOB']].sum()

# Incluindo a descrição simplificada 
line['DESCRIÇÃO SIMPLIFICADA'] = line['NCM'].map(dict_ncm12)

# Plotando os dados
fig = px.line(line, x="ANO", y="VALOR_FOB", color='DESCRIÇÃO SIMPLIFICADA',color_discrete_sequence=px.colors.qualitative.Set3)
fig.update_layout(title={'text':'EVOLUÇÃO - 2010 a 2021','y':0.925,'x':0.3})
fig.show()

In [ ]:
af = exp.groupby(['NCM','PAIS','ANO'], as_index=False)[['VALOR_FOB']].sum()
af['NCM']= af['NCM'].astype(str)
af['NCM_2'] = af['NCM'].str[:2]
af = af.groupby(['NCM_2','PAIS','ANO'], as_index=False)[['VALOR_FOB']].sum()
af

In [ ]:
ncm_cap.info()

In [ ]:
ncm_cap = pd.read_csv('/content/drive/My Drive/cap ncm8.csv',sep=';')
ncm_cap['Cap']=ncm_cap['Cap'].astype(str)
ncmcap_dict = {ncm_cap.iloc[i,0]:ncm_cap.iloc[i,1] for i in range(ncm_cap.shape[0])}
ncmcap_dict

In [ ]:
af['DESCRIÇÃO CAP']=af['NCM_2'].map(ncmcap_dict)
af

In [ ]:
fig = px.treemap(af,path=['ANO','DESCRIÇÃO CAP','PAIS'], values='VALOR_FOB')
fig.show()

<font size='4'> Modais </font>

In [ ]:
exp_2021.columns

In [ ]:
# Modais mais utilizado na exportação
# Top 3 - representam 99% da pauta de exportação
# Top 1 - o modal marítimo representa 88%!
modal=exp_2021.groupby(['MODAL'])[['VALOR_FOB']].sum().sort_values(['VALOR_FOB'], ascending=False)
modal['%']=modal.VALOR_FOB/modal.VALOR_FOB.sum()*100
modal.reset_index(inplace=True)
ac = [modal.iloc[0,2]]
for i in range(modal.shape[0]-1):
    ac.append(ac[i]+modal.iloc[i+1,2])
modal['% Acu'] = ac
modal

In [ ]:
modalxpaises = pd.crosstab(exp_2021['PAIS'], exp_2021['MODAL'],values=exp_2021['VALOR_FOB'], aggfunc=np.sum)
modalxpaises.sort_values(['AEREA'],ascending=False).head(10)